In [37]:
import sys

from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sys.path.append("/home/aseliverstov/projects/brain_signals/experiments/src")
from src.dataset import EEG

In [38]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/dataset")
intencities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
n_channels = 31

dataset = EEG(data_dir=data_dir, intencities=intencities, n_channels=n_channels)
X, y = dataset.load_to_dataframe(users=[1, 2, 3, 4, 5], signals=["delta", "theta", "alpha", "beta", "gamma"])

# Replication

In [ ]:
import pandas as pd

ids = []
for user in [1, 2, 3, 4, 5]:
    for signal_type in ["delta", "theta", "alpha", "beta", "gamma"]:
        ids.append(dataset._process_signal_type_participant(user, signal_type).iloc[:, :5])
ids = pd.concat(ids, axis=0).reset_index(drop=True)

ids.head()

,user,intensity,channel,is_left_channel,signal_type
0,1,0.1,0,1,delta
1,1,0.1,1,0,delta
2,1,0.1,2,1,delta
3,1,0.1,3,0,delta
4,1,0.1,4,1,delta


In [40]:
train_ids = (ids["signal_type"] != "delta").to_numpy()
test_ids = (ids["signal_type"] == "delta").to_numpy()

X_train, y_train = X[train_ids], y[train_ids]
X_test, y_test = X[test_ids], y[test_ids]

In [41]:
import numpy as np
from scipy.stats import kurtosis, skew

def calculate_summary_features(arr, n_windows=30):
    features = []

    cur_arr = np.array_split(arr, n_windows, axis=1)
    for window in cur_arr:
        features.append(np.mean(window, axis=1))
        features.append(np.mean(np.abs(window), axis=1))
        features.append(np.std(window, axis=1))
        features.append(np.sqrt(np.mean(np.square(window), axis=1)))
        features.append(kurtosis(window, axis=1, fisher=True))
        features.append(skew(window, axis=1))

    features = np.column_stack(features)    
    return features

In [42]:
X_train = calculate_summary_features(X_train)
X_test = calculate_summary_features(X_test)

X_train.shape

(6200, 180)

In [43]:
model = Sequential()
model.add(Dense(64, input_shape= (X_train.shape[1],), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=10, batch_size=10, verbose=1)

/home/aseliverstov/.cache/pypoetry/virtualenvs/brain-signals-_5HxkjSc-py3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │        11,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 10)             │           170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,690 (61.29 KB)

 Trainable params: 15,690 (61.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1247 - loss: 2.4085 - val_accuracy: 0.1484 - val_loss: 2.8059
Epoch 2/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1835 - loss: 2.1999 - val_accuracy: 0.2052 - val_loss: 3.1018
Epoch 3/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2428 - loss: 2.0939 - val_accuracy: 0.2065 - val_loss: 3.1391
Epoch 4/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2825 - loss: 2.0093 - val_accuracy: 0.2581 - val_loss: 3.8312
Epoch 5/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3298 - loss: 1.9132 - val_accuracy: 0.2245 - val_loss: 4.2321
Epoch 6/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.3669 - loss: 1.8392 - val_accuracy: 0.2768 - val_loss: 4.9189
Epoch 7/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3824 - loss: 1.7790 - val_accuracy: 0.2819 - val_loss: 5.5703
Epoch 8/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4327 - loss: 1.6951 - val_accuracy: 0.